### (unpack the data.rar file)

# Logistic regression

useful resources:

[1] [Cross entropy for dummies](https://towardsdatascience.com/cross-entropy-for-dummies-5189303c7735)

[2] [Understanding logistic regression](https://towardsdatascience.com/understanding-logistic-regression-step-by-step-704a78be7e0a)

[3] [Cross entropy log loss and intuition behind it](https://towardsdatascience.com/cross-entropy-log-loss-and-intuition-behind-it-364558dca514)

[4] [Cross entropy (see the section "Relation with log-likelihood")](https://en.wikipedia.org/wiki/Cross_entropy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import functools
import pandas as pd
from typing import Union

plt.rc('font', **{'size' : 18})

In [ ]:
import tableprint as tab

## Introduction

Linear regression 'matches' a linear (polinomial) function using a set of data $X$, on the whole domain $\mathbb{R}$, $linreg(x) : \mathbb{R}^m \rightarrow \mathbb{R}$. The logistic regression has the same domain, however the codomain is way more limited, namely $logreg(x) : \mathbb{R}^n \rightarrow (0, 1)$. It tries to predict the probability that the element $x \in X$ is to be part of the positive class.

This probability is marked as $P(y=1|x,\theta)$, and we take it as being the probability associated to the feedback $x^T \theta$ calculated by the classic regression, under the circumstance that we know the features $X$ and the parameters $\theta$ of the model.

The ideea is that for every entry $x$, the logistic regression model pairs a probability. We will show how we choose the function that calculates the probability using the result of the linear regression $x^T \theta$.

We start from a function that has as parameter a probability and that maps the interval $(0, 1)$ on the whole real axis $\mathbb{R}$. We observe how the function $f_1(p) = \frac{p}{1-p}$ maps the probability in $\mathbb{R}+$, and if we apply the logarithm, the function $f_2(p) = log\left(\frac{p}{1-p}\right)$ maps the interval $(0, 1)$ on the whole real axis $\mathbb{R}$:

In [ ]:
f1 = lambda x: x / (1 - x)
f2 = lambda x: np.log(x / (1 - x))
x:np.array = np.linspace(1e-4, 1-1e-4, 100)

fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
ax[0].plot(x, f1(x)) 
ax[1].plot(x, f2(x))
ax[0].set_ylabel(r'$f_1(p) = \frac{p}{1-p}$') 
ax[0].grid()
ax[1].set_xlabel('p') 
ax[1].set_ylabel(r'$f_2(p) = log\left(\frac{p}{1-p}\right)$') 
ax[1].grid()
plt.show()

Observe that if the answer given by the regression is 0 ($f_2(p) = 0$), the probability associated is 0.5. We want the response calculated by the regression (which can be any value on the real axis) to be equal with this function of probability:

$$h_{\theta}(x) = x^T \theta = log\left(\frac{p}{1-p}\right)$$

Onwards, with a few algebraic conversions, we can prove that:

$$\hat{y} = P(y=1|x, \theta) = p = \frac{1}{1 + e^{-h_{\theta}(x)}}$$

Thus we have a function that maps $x$ and the parameters of the model $\theta$ in a probability. The function is named as logistic function.

In reality, we have the dataset $X$, and a binary value $y_i \in \{0, 1\}$ associated to every $x_i$, $i=1 \dots m$. Actually, the dataset $X$ is described by a very simple probability distribution, a distribution in which the probabilities can take only the values 0 or 1. If we put in the values $x_i$ on the x axis and the values $y_i$ on the y axis, the distribution graph would be very "damaged" - after all, y takes only 2 values.

We want the model defined by the coefficients $\theta$ to 'fit' as good as possible over this initial given distribution - perhaps the "landscape" of the distribution function of the model is not so rough anymore, but smoother (continuous). Of course this example is exaggerated, in reality the entry space $X$ is $m$ dimansional, not 1-dimensional as we have assumed here that we imagine the representation of probability distribution.

However, in order to compare 'how good' the distribution of the model that we learn fits with the initial distribution, we need a measure of these distributions. For this we will introduce the notion of entropy as measure of information.

## The binomial logistic regression
The cost function for the logistic regression is given by the binary cross-entropy, written in a vectorial way as such:

$$
J(\theta) = -\frac{1}{m} \left(\mathbf{y}^t \cdot \ln \hat{\mathbf{y}} + (1-\mathbf{y})^t \cdot \ln ( \mathbb{1}_m - \hat{\mathbf{y}}) \right) 
$$

The gradient of the cost function is:

$$
\nabla_{\theta} J(\theta) = \frac{1}{m} \mathbf{X}^t (\hat{\mathbf{y}} - \mathbf{y})
$$

The change of weight from the vector $\boldsymbol{\theta}$ is made at every iteration (epoch) with:
$$
\boldsymbol{\theta} = \boldsymbol{\theta} - \alpha\frac{1}{m} \mathbf{X}^t (\hat{\mathbf{y}} - \mathbf{y})
$$
with $\alpha > 0$ as learning rate. 

https://kisaragiry.medium.com/binomial-logistic-regression-math-explained-c2569cdbd2c5
### Loading the dataset

In [ ]:
path_train: str = './data/mnist_train.csv'
path_test: str = './data/mnist_test.csv'

train_set: np.ndarray = pd.read_csv(path_train, header=None).values
test_set: np.ndarray = pd.read_csv(path_test, header=None).values

assert(train_set.shape) == (60000, 785)
assert(test_set.shape) == (10000, 785)

In [ ]:
# train_set and test_set are matrixes that contain on the first column the class (a digit from 0 to 9),
#  and the image is being held starting from the column 1 to the end.
# take into train_x only the images and into train_y only the class, 
# and do the same for test_x and test_y

train_x, train_y = train_set[:, 1:], train_set[:, 0]
test_x, test_y = test_set[:, 1:], test_set[:, 0]

assert train_x.shape == (60000, 784)
assert train_y.shape == (60000,)
assert test_x.shape == (10000, 784)
assert test_y.shape == (10000,)

Lets see the dataset. We will observe the first 16 lines from the training set:

In [ ]:
def show_samples(x_set: np.ndarray, y_set: np.array) -> None:
    size: int = x_set.shape[0]
    
    fig, ax = plt.subplots(size // 4, 4, figsize=(20, 10))
    for k in range(size):
        row, col = k // 4, k % 4
        
        # Make those columns into a array of 8-bits pixels
        # The pixel intensity values are integers from 0 to 255
        pixels = np.array(x_set[k], dtype='uint8')    
        
        # Reshape the array into 28 x 28 array (2-dimensional array)
        n = int(np.sqrt(len(pixels)))
        assert n**2 == len(pixels)
        pixels = pixels.reshape(n, n)
        ax[row, col].imshow(pixels, cmap='gray')
        ax[row, col].set_title('Label {label}'.format(label=y_set[k]))
        ax[row, col].axis('off')

    plt.show()
    
show_samples(train_x[:16, :], train_y[:16])

For the binomial regression we care about classifying only the relevant images of two classes, like the digits '0' and '1' (for now). We will define the sets that 'crop' only these two classes from the given sets:

In [ ]:
# filter from the big sets only the samples suitable for the digits 0 and 1
# determine the logical vectors with True for the indexes for which the classes from the vectors y are 0 or 1 and False otherwise.

labels_0_1_train: np.ndarray = np.logical_or(train_y == 0, train_y == 1)
labels_0_1_test: np.ndarray = np.logical_or(test_y == 0, test_y == 1)

print(labels_0_1_train, '\n', labels_0_1_test, '\n')

assert labels_0_1_train.shape == (60000,)
assert labels_0_1_test.shape == (10000,)

assert labels_0_1_train.sum() == 12665
assert labels_0_1_test.sum() == 2115

# filter from the big sets only the samples suitable for the digits 0 and 1
# you can use reshape for the vectors y
# use the logical vectors labels_0_1_train and labels_0_1_test for the filtering
train_x_bin, train_y_bin = train_x[labels_0_1_train,
                                   :], train_y[labels_0_1_train].reshape(-1, 1)
test_x_bin, test_y_bin = test_x[labels_0_1_test,
                                :], test_y[labels_0_1_test].reshape(-1, 1)

print(train_x_bin, '\n\n', train_y_bin, '\n')
print(test_x_bin, '\n\n', test_y_bin)

assert train_x_bin.shape == (12665, 784)
assert train_y_bin.shape == (12665, 1)
assert test_x_bin.shape == (2115, 784)
assert test_y_bin.shape == (2115, 1)


Just like the linear regression, the first column needs to be made of ones:

In [ ]:
def add_ones_column(x: np.ndarray) -> np.ndarray:
    """ Returns a matrix with first column filled with 1 and the other columns being x's columns. """
    return np.concatenate((np.ones(shape = (x.shape[0], 1)), x), axis = 1)

train_x_bin_ext: np.ndarray = add_ones_column(train_x_bin)
test_x_bin_ext: np.ndarray = add_ones_column(test_x_bin)

assert train_x_bin_ext.shape == (12665, 785)
assert test_x_bin_ext.shape == (2115, 785)
assert np.all(train_x_bin_ext[:, 0] == 1)
assert np.all(test_x_bin_ext[:, 0] == 1)

The features need to be normalized, the maximum value being 255. The normalization follows as the resulting features to be between the interval \[0, 1\], so we divite to the maximum value.

In [ ]:
def normalize(x: np.ndarray) -> np.ndarray:
    """ Normalization means division by 255.
    Args:
        x: feature matrix, shape m * n. It will not be changed by this code.
    Returns:
        matrix with scaled values between 0 and 1, of same shape"""
    return x/255

train_x_bin_ext: np.ndarray = add_ones_column(normalize(train_x_bin))
test_x_bin_ext: np.ndarray = add_ones_column(normalize(test_x_bin))

assert train_x_bin_ext.shape == (12665, 785)
assert test_x_bin_ext.shape == (2115, 785)
assert np.all(train_x_bin_ext[:, 0] == 1)
assert np.all(test_x_bin_ext[:, 0] == 1)
assert np.all(train_x_bin_ext <= 1)
assert np.all(test_x_bin_ext <= 1)
assert np.all(train_x_bin_ext >= 0)
assert np.all(test_x_bin_ext >= 0)

We calculate the sigmoid function, $sigmoid(z) = \frac{1}{1 + e^{-z}}$ respectively $\hat y = h(x, \theta) = sigmoid \left( X  \theta \right)$:

In [ ]:
def sigmoid(z: Union[float, np.ndarray]) -> Union[float, np.ndarray]:
    return 1/(1+np.exp(-z))

assert sigmoid(0) == 0.5
assert np.abs(sigmoid(1) - 0.731058) < 1e-6

def h(x: Union[float, np.ndarray], theta: Union[float, np.ndarray]) -> Union[float, np.ndarray]:
    return sigmoid(np.dot(x, theta))

assert np.abs(h(np.array([1., 1., 1., 0]), np.array([1., 0., 1., 1.])) - 0.880797) < 1e-6

We calculate the cost function with adjustment this time (careful, the coefficient $\theta_0$ does not adjust):

\begin{align}
J(\boldsymbol\theta) & = \underbrace{-\frac{1}{m} \sum\limits_{i=1}^m \left[y^{(i)}\cdot \ln h_{\boldsymbol\theta}(\mathbf{x}^{(i)}) + (1-y^{(i)})\cdot \ln (1- h_{\boldsymbol\theta}(\mathbf{x}^{(i)})) \right]}_{\text{error of quality}} 
\\
& + \underbrace{\frac{\lambda}{2} \sum\limits_{j=1}^n \theta_j^2}_{\text{term of adjustment}}
%\label{eq:JCrossEntropyRegularizare}
\\
& = 
\underbrace{-\frac{1}{m} \sum\limits_{i=1}^m \left[y^{(i)}\cdot \ln \hat y^{(i)}  + (1-y^{(i)})\cdot \ln (1- \hat y^{(i)}) \right]}_{\text{error of quality}} 
\\
& + \underbrace{\frac{\lambda}{2} \sum\limits_{j=1}^n \theta_j^2}_{\text{term of adjustment}}
\\
& = -\frac{1}{m} \left(\mathbf{y}^t \cdot \ln\hat{\mathbf{y}}  + (\mathbb{1}-\mathbf{y})^t \cdot \ln(\mathbb{1}-\hat{\mathbf{y}})\right) + \frac{\lambda}{2} \| \boldsymbol\theta[1:]  \|^2_2
\end{align}

where $\boldsymbol\theta[1:]$ is the vector made from all the components of $\boldsymbol\theta$ excluding the first one, and $\| \mathbf{v} \|_2$ is the Euclidean norm of the vector $\mathbf{v}$.

In [ ]:
def cost(x: np.ndarray, y: np.ndarray, theta: np.ndarray, lmbda: float) -> float:
    """ Cost function includes also regularization
    Args:
        x: the feature matrix, dimension m x (n+1)
        y: the evidence vector, dimension m x 1
        theta: the vector of coefficients, dimension (n+1) x 1
    Returns:
        the cost, as a scalar"""
    assert x.shape[1] == theta.shape[0]
    assert x.shape[0] == y.shape[0]
    assert theta.shape[1] == y.shape[1] == 1
    assert lmbda >= 0
    # calculating y_hat:
    y_hat: any = h(x, theta)
    # calculating the error of quality:
    m: int = x.shape[0]
    j1: float = -1/m * np.sum(y * np.log(y_hat) + (1-y) * np.log(1-y_hat))
    # calculating the error of adjustment:
    j2: float = lmbda/(2) * np.sum(theta[1:]**2)
    print(j1 + j2)
    return j1 + j2
    

np.random.seed(11)
n: int = train_x_bin_ext.shape[1]
theta: np.ndarray = np.random.randn(n).reshape(n, 1)
assert np.abs(cost(train_x_bin_ext, train_y_bin, theta=theta, lmbda=0.2) - 79.815566) < 1e-6

We calculate the gradient, using the expression defined previously, taking into account the term of adjustment:

$$
\nabla_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) = \frac{1}{m} \mathbf{X}^t(\hat{\mathbf{y}} - \mathbf{y}) + \lambda (0, \theta_1, \dots, \theta_n)
$$

In [ ]:
def grad(x: np.ndarray, y: np.ndarray, theta: np.ndarray, lmbda: float) -> np.array:
    """ Computation of gradient with regularization term
    
    Args:
        x: the feature matrix, dimension m x (n+1)
        y: the evidence vector, dimension m
        theta: the vector of coefficients, dimension n+1
        lmbda: the coefficient of adjustment
    Returns:
        the gradient,as vector of n+1 elements"""
    m: int = x.shape[0]
    y_hat: any = h(x, theta)
    g1: np.array = (1/m) * np.dot(x.T, (y_hat - y)) # the gradient of quality, without adjustment
    
    theta_simple = theta.copy()
    theta_simple[0] = 0
    # the gradient related to the term of adjustmentt
    g2: np.array = lmbda * theta_simple 
    
    g: np.array = g1 + g2
    assert g.shape == theta.shape
    return g

np.random.seed(11)
n: int = train_x_bin_ext.shape[1]
theta: np.array = np.random.randn(n).reshape(n, 1)
res: np.ndarray = grad(train_x_bin_ext, train_y_bin, theta=theta, lmbda=0.2)
assert res.shape == (785, 1)

The training algorithm will store the cost associated to every epoch into a list:

In [ ]:
def compute_accuracy(x_set: np.ndarray, y_set: np.ndarray, theta: np.ndarray) -> float:
    pred: np.array = ((h(x_set, theta) >= 0.5) * 1 == y_set)
    return 100.0 * sum(pred) / pred.shape[0]

In [ ]:
# Set learning rate
eta: float = 0.2

# Set regularization coefficient
lmbda: float = 0.5

# In x, we have m instances of n features each
# Create theta as a vector (n x 1)
n: int = np.shape(train_x_bin_ext)[1]
theta: np.array = np.random.randn(n).reshape(n, 1)

# Do the training
epochs: int = 40
values: list = []
accurracies: list = []
for i in range(epochs):
    theta -= eta * grad(train_x_bin_ext, train_y_bin, theta, lmbda)
    acc = compute_accuracy(train_x_bin_ext, train_y_bin, theta)
    values.append(cost(train_x_bin_ext, train_y_bin, theta, lmbda))
    accurracies.append(acc)
    
print("last cost: %g" % values[-1])

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
ax[0].plot(range(len(values)), values)
ax[0].set_xlabel('epoch') ; ax[0].set_ylabel('cost')
ax[0].grid()
ax[1].plot(range(len(accurracies)), accurracies)
ax[1].set_xlabel('epoch') ; ax[1].set_ylabel('accurracy [%]')
ax[1].grid()
plt.show()

In [ ]:
# we calculate the accuracy for the test set
# this assumes we count how many predictions fit with the reality and to express this idea percentage-wise
pred = (h(test_x_bin_ext, theta) >= 0.5) * 1 == test_y_bin
print("accuracy: %2.2f%% for %d patterns" % (accurracies[-1] , test_x_bin_ext.shape[0]))

# calculating confusion matrix
# true positive: y = 1 and pred = 1
# true negative: y = 0 and pred = 0
# false positive: y = 0 and pred = 1
# false negative: y = 1 and pred = 0
pred = (h(test_x_bin_ext, theta) >= 0.5) * 1
tp = np.sum(np.logical_and(pred == 1, test_y_bin == 1))
tn = np.sum(np.logical_and(pred == 0, test_y_bin == 0))
fp = np.sum(np.logical_and(pred == 1, test_y_bin == 0))
fn = np.sum(np.logical_and(pred == 0, test_y_bin == 1))

headers: list = ['Confusion Matrix', 'pred: 0', 'pred: 1', 'pred: all'] 
table: list = [
    ['actual: 0', tn, fp, tn + fp],
    ['actual: 1', fn, tp, fn + tp],
    ['actual: all', tn + fn, fp + tp, tn + fn + fp + tp]]
tab.table(table, headers, width=16)

## Multinomial logistic regression

In [ ]:
# classes
k: int = 10

# We add the ones column to features
train_x_all_ext = add_ones_column(normalize(train_x))
test_x_all_ext = add_ones_column(normalize(test_x))

assert train_x_all_ext.shape == (60000, 785)
assert test_x_all_ext.shape == (10000, 785)
assert np.all(train_x_all_ext[:, 0] == 1)
assert np.all(test_x_all_ext[:, 0] == 1)
assert np.all(train_x_all_ext <= 1)
assert np.all(test_x_all_ext <= 1)
assert np.all(train_x_all_ext >= 0)
assert np.all(test_x_all_ext >= 0)

In [ ]:
def one_hot(val: int, classes: int) -> np.ndarray:
    """Achieves 'one-hot encoding', the conversion of an integer at a binary array, 
    that has 1 only on the position specified by the 'val' value
    Args:
        val: the class that needs to be encoded, an integer between {0, 1, ... K-1}
        classes: the number of K classes
    Returns:
        an array of zeroes of length K, where only on the position 'val' we have a value of 1"""
    assert 0 <= val < classes
    result = np.zeros(classes)
    result[val] = 1
    result = result.reshape(1, classes)
    assert result.shape == (1, classes)
    return result

assert np.all(one_hot(7, k) == np.array([[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]]))
assert np.all(one_hot(3, k) == np.array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]]))

train_y_all = np.concatenate([one_hot(int(i), k) for i in train_y])
test_y_all = np.concatenate([one_hot(int(i), k) for i in test_y])
assert train_y_all.shape == (60000, 10)
assert test_y_all.shape == (10000, 10)
assert np.all((train_y_all != 0) == (train_y_all == 1))

The product $X \theta$ among the matrix X of dimension (m, n) and $\theta$ of dimension (n, k) will have the dimension (m, k):

In [ ]:
def prod(x: np.ndarray, theta: np.ndarray) -> np.ndarray:
    """Product between X of shape (m x n) and theta of shape (n x k)
    Args:
        x: the features, of dimension m x n
        theta: parameters, of dimension n x k
    Returns:
        their product of dimension m x k"""
    return np.dot(x, theta)

m, n = train_x_all_ext.shape
np.random.seed(11)
theta = np.random.randn(n, k)
assert prod(train_x_all_ext, theta).shape == (m, k)

The function $softmax()$ will have the same dimensions (m, k) and needs to give the sum 1 on every column. Its calculation can be written in a more compact way as:

$$softmax(X, \theta) = \frac{e^{X \theta}}{e^{X \theta} \cdot \mathbb{1}_k}$$

The denominator term, $e^{X \theta} \cdot \mathbb{1}_k$, is not a matrix anymore, but a vector of dimension (m, 1) (essentially the computation of the sum on every lane is done). For the accomplishment of the division we do the broadcast operation.

In [ ]:
def softmax(x: np.ndarray, theta: np.ndarray) -> np.ndarray:
    """"
    The calculation of the softmax function
    Args:
        x: features, of dimension m x n
        theta: parameters, of dimension n x k
    Returns:
        their product, of dimension m x k"""
    assert x.shape[1] == theta.shape[0]
    a: float = np.exp(prod(x, theta))
    ones: np.array = np.ones((k, 1))
    return a / prod(a, ones)

m, n = train_x_all_ext.shape
np.random.seed(11)
theta: np.ndarray = np.random.randn(n, k)
smax: np.ndarray = softmax(train_x_all_ext, theta)
assert smax.shape == (m, k)
assert np.all((smax.sum(axis=1) - 1) < 1e-12)

The cost function that includes the adjustment can be written in a more compact way as (note that the index $i$ starts at 1):

$$J(\theta, \lambda) = -\frac{1}{m} \mathbb{1}_m^T \left\{ Y \odot \log[softmax(X \theta)] \mathbb{1}_k \right\} + \frac{\lambda}{2} \sum_{i=1}^{n-1}\sum_{j=0}^{k-1} \theta_{i,j}^2 = -\frac{1}{m} \mathbb{1}_m^T \left\{ Y \odot \log[softmax(X \theta)] \mathbb{1}_k \right\} + \frac{\lambda}{2} \|\Theta[1:, :]\|_F^2$$

where forr a matrix $A$ of type $m\times n$, $\| A \|_F$ is the Forbenius norm: $\| A \|_F = \sqrt{\sum\limits_{i=0}^{m-1}\sum\limits_{j=0}^{n-1} |a_{ij}|^2}$

In [ ]:
# def sigma_sum(start: int, end: int, expression: any) -> any:
#     return sum(expression(i) for i in range(start, end))

def cost(x: np.ndarray, y:np.ndarray, theta: np.ndarray, lmbda: float) -> float:
    """The cost includes the adjustment
    Args:
        x: the features of dimension m x n
        y: the classes, of dimension m x k
        theta: parameters, of dimension n x k
        lmbda: the parameter of adjustment, scalar
    Returns:
        the cost as scalar"""
    # not working!!!!!!!!!!!!!!!!!!!!!!!
    # m=x.shape[0]
    # a = (-1/m)*np.ones((m,1)).T @ (y * np.log(softmax(x, theta)) @ np.ones((k, 1)))
    # b = (lmbda/(2*m))*np.sum(np.square(theta[1:]))
    # # b = (lmbda/(2*m))*np.ones((n,1)).T @ np.ones((n,1))
    # print(a + b) 
    # return a + b
    return -np.sum(y * np.log(softmax(x, theta))) / x.shape[0] + lmbda * np.sum(theta ** 2) / (2 * x.shape[0])

m, n = train_x_all_ext.shape
np.random.seed(11)
theta = np.random.randn(n, k)
assert (cost(train_x_all_ext, train_y_all, theta=theta, lmbda=0.2) - 804.384938) < 1e-6

The gradient is calculated as:

$$\nabla_{\theta}J = - \frac{1}{m} X^T \left[Y - softmax(X \theta) \right] + \lambda \left[ \mathbb{0}_n, \theta_{1 \dots k-1} \right]$$

where the matrix $\left[ \mathbb{0}_n, \theta_{1 \dots k-1} \right]$ still has the dimension (n, k), just like $\theta$, but the first line is zero.

In [ ]:
def deltas(x: np.ndarray, y: np.ndarray, theta: np.ndarray, lmbda: float) -> np.ndarray:
    """Calculates the gradient
    Args:
        x: the features, of dimension m x n
        y: the classes, of dimension m x k
        theta: the parameters, of dimension n x k
        lmbda: the parameter of adjustment, scalar
    Returns:
        the matrix of gradients, of theta's dimension (n x k)"""
    
    a = (-1/m) * (x.T @ (y - softmax(x, theta)))
    theta_copy = theta.copy()
    theta_copy[0] = 0
    b = lmbda * theta_copy
    return a + b

m, n = train_x_all_ext.shape
np.random.seed(11)
theta = np.random.randn(n, k)
grad = deltas(train_x_all_ext, train_y_all, theta=theta, lmbda=0.2)
assert grad.shape == (n, k)
print(grad.sum()+6.0286086)
assert (grad.sum() + 6.0286086) < 1e-6

In [ ]:
from sklearn.metrics import accuracy_score

def calculate_accurracy(set_x, set_y, theta):
    a = softmax(set_x, theta)
    b = np.argmax(a, axis=1)
    c = np.argmax(set_y, axis=1)
    return 100.0 * accuracy_score(b, c)     # same output with or without 100.0
    

In [ ]:
# number of classes
k = 10

lmbda, alpha = 0.05, 0.65
m, n = train_x_all_ext.shape
np.random.seed(11)
theta = np.random.randn(n, k)

epochs: int = 300
values: list = []
accurracies: list = []
for i in range(epochs):
    theta -= alpha * deltas(train_x_all_ext, train_y_all, theta, lmbda)
    if (i % 10 == 0):
        values.append(cost(train_x_all_ext, train_y_all, theta, lmbda))
        accurracies.append(calculate_accurracy(test_x_all_ext, test_y_all, theta))
        print("epoch: ", i, "cost: ", values[-1])
        lmbda *= 0.9
    
print("last costs: %g" % values[-1])

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
ax[0].plot([x * 10 for x in range(len(values))], values, 'o-')
ax[0].set_xlabel('epoch') ; ax[0].set_ylabel('cost')
ax[0].grid()
ax[1].plot([x * 10 for x in range(len(accurracies))], accurracies, 'o-')
ax[1].set_xlabel('epoch') ; ax[1].set_ylabel('accurracy [%]')
ax[1].grid()
plt.show()

In [ ]:
# we calculate the accuracy of the test set
# this assumes we count how many predictions fit with the reality and to express this idea in a procentually-wise way
pred = softmax(test_x_all_ext, theta)
actual = np.argmax(pred, axis=1)
equalities = np.sum(pred == actual)
print("Test accuracy: %2.2f%%" % (100.0 * accuracy_score(actual, np.argmax(test_y_all, axis=1))))

In [ ]:
# we calculate the vectors of the predictions as well as the vector of the true values
pred: np.ndarray = np.argmax(softmax(test_x_all_ext, theta), axis=1)
actual: np.ndarray = np.argmax(test_y_all, axis=1)

# confusion matrix will have at the intersection line/column how many samples from the given class by the number
# of the line have been predicted as being part of the class given by the column number
conf_matrix: np.ndarray = np.zeros((k, k), dtype=np.int32)
for i in range(len(pred)):
    conf_matrix[actual[i], pred[i]] += 1

assert len(conf_matrix) == k
assert (sum(len(row) for row in conf_matrix)) == k ** 2

    
headers = ['CnfMat'] + [f'pr: {x}' for x in range(k)] + ['all a'] 
table = []
for i in range(k):
    table.append([f'act: {i}'] + list(conf_matrix[i]) + [sum(conf_matrix[i])])
table.append(['all p'] + [sum(conf_matrix[:, i])
             for i in range(k)] + [sum(sum(conf_matrix))])

tab.table(table, headers, width=6)